In [1]:
!pip install opendatasets

In [8]:
import opendatasets as od
import cv2
import numpy as np

od.download(
    "https://www.kaggle.com/competitions/spaceship-titanic/data"
)

Skipping, found downloaded files in "./spaceship-titanic" (use force=True to force download)


In [51]:
import pandas as pd
train=pd.read_csv("/content/spaceship-titanic/train.csv")
test=pd.read_csv("/content/spaceship-titanic/test.csv")


In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [21]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [52]:
# drop verbose features
train=train.drop(['Name','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Age','Cabin','VIP'],axis=1)

In [23]:
train.isnull().sum()

PassengerId      0
HomePlanet     201
CryoSleep      217
Destination    182
Transported      0
dtype: int64

In [53]:
from sklearn.impute import SimpleImputer
mssi=SimpleImputer(strategy='most_frequent')

train['HomePlanet']=mssi.fit_transform(train[['HomePlanet']])
train['CryoSleep']=mssi.fit_transform(train[['CryoSleep']])
train['Destination']=mssi.fit_transform(train[['Destination']])

In [54]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  8693 non-null   object
 1   HomePlanet   8693 non-null   object
 2   CryoSleep    8693 non-null   object
 3   Destination  8693 non-null   object
 4   Transported  8693 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 280.3+ KB


In [55]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
train['HomePlanet']=oe.fit_transform(train[['HomePlanet']])

In [56]:
train['CryoSleep']=oe.fit_transform(train[['CryoSleep']])
train['Destination']=oe.fit_transform(train[['Destination']])
train['Transported']=oe.fit_transform(train[['Transported']])

In [100]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train.drop(['Transported'],axis=1),train['Transported'],test_size=0.2,random_state=0)

In [105]:
from sklearn.ensemble import AdaBoostClassifier
ac=AdaBoostClassifier(n_estimators=50,learning_rate=0.2,random_state=0)
ac.fit(x_train,y_train)
y_predicted=ac.predict(x_test)

In [106]:
from sklearn.metrics import accuracy_score
accuracy_score(y_predicted,y_test)

0.7084531339850488

In [107]:
test=test[['PassengerId','HomePlanet','CryoSleep','Destination']]

test['HomePlanet']=mssi.fit_transform(test[['HomePlanet']])
test['CryoSleep']=mssi.fit_transform(test[['CryoSleep']])
test['Destination']=mssi.fit_transform(test[['Destination']])
test['HomePlanet']=oe.fit_transform(test[['HomePlanet']])
test['CryoSleep']=oe.fit_transform(test[['CryoSleep']])
test['Destination']=oe.fit_transform(test[['Destination']])
y=ac.predict(test)

In [108]:
y=y.astype(str)
y[y=='1.0']='True'
y[y=='0.0']='False'
answer=pd.DataFrame({
    'PassengerId':test['PassengerId'],
    'Transported':y
})
answer.to_csv('./answer.csv',sep=',',index=False,header=True)